# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 1 from hw3-t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def residual(graph, flow):
    Gf = nx.DiGraph()
    for i in graph.nodes():
        Gf.add_node(i)
    for (s1, s2) in graph.edges():
        if flow[s1][s2] < graph[s1][s2]['capacity']:
            Gf.add_edge(s1, s2 ,c = graph[s1][s2]['capacity'] - flow[s1][s2], label = 'forward')
        if flow[s1][s2] > 0:
            Gf.add_edge(s2, s1, c = flow[s1][s2], label = 'backward')
    return Gf
def st_path(graph, source, sink):
    #BFS
    discovered = {}
    parent = {}
    dist = {}
    for i in graph.nodes():
        discovered[i] = False
        parent[i] = None
        dist[i] = 922337203685477580
    q = []
    q.append(source)
    discovered[source] = True
    dist[source] = 0
    while q:
        u = q.pop(0)
        for v in graph.neighbors(u):
            if discovered[v] == False:
                discovered[v] = True
                dist[v] = dist[u] + 1
                parent[v] = u
                q.append(v)
    if parent[sink] == None:
        return (922337203685477580, None)
    p = sink
    path = []
    min_c = 922337203685477580
    while 1:
        pp = parent[p]
        if pp == None:
            break
        cc = graph.edge[pp][p]['c']
        if cc < min_c:
            min_c = cc
        path.append((pp, p))
        p = pp
    path.reverse()
    return (min_c, path)
        
def aug(graph, flow, path, min_c):
    for (s1, s2) in path:
        if graph.edge[s1][s2]['label'] == 'forward':
            flow[s1][s2] = flow[s1][s2] + min_c
        else:
            f1ow[s2][s1] = flow[s2][s1] - mic_c
    return flow
        
def max_flow(graph, source, sink):
    f = {}
    for i in graph.nodes():
        f[i] = {}
    for (s1, s2) in graph.edges():
        f[s1][s2] = 0
    Gf = residual(graph, f)
    
    (min_c, s) = st_path(Gf, source, sink)
    while s:
        #print s
        f1 = aug(Gf, f, s, min_c)
        f = f1
        Gf = residual(graph, f)
        (min_c, s) = st_path(Gf, source, sink)
    fv = 0
    for s in graph.neighbors(source):
        fv += f[source][s]
    return fv, f
    #s = st_path(Gf)

In [5]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    flow = {}
    for (s1, s2) in graph.edges():
        t = {s2 : -1}
        flow[s1] = t
    sum_supply = 0
    sum_demand = 0
    S = []
    T = []
    for i in graph.nodes():
        if graph.node[i]['demand'] > 0:
            T.append(i)
            sum_demand += graph.node[i]['demand']
        else:
            S.append(i)
            sum_supply -= graph.node[i]['demand']
    if sum_demand != sum_supply:
        raise nx.NetworkXUnfeasible("supply is not equal to demand!")
        
    graph.add_nodes_from(["newsource", "newsink"])
    for s in S:
        graph.add_edge("newsource", s, capacity = -1 * graph.node[s]['demand'])
    for t in T:
        graph.add_edge(t, "newsink", capacity = graph.node[t]['demand'])
    print "--------"
    (fv, fd) = max_flow(graph, "newsource", "newsink")
    #(fv, fd) = nx.maximum_flow(graph, "newsource", "newsink")
    
    for s in S:
        graph.remove_edge("newsource", s)
    for t in T:
        graph.remove_edge(t, "newsink")
    graph.remove_nodes_from(["newsource", "newsink"])
    if fv < sum_demand:
        raise nx.NetworkXUnfeasible("No feasible circulation")
    for (s1, s2) in graph.edges():
        flow[s1][s2] = fd[s1][s2]
   
    return flow

#f = flow_with_demands(G)

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [6]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    netflow = {}
    for i in flow:
        netflow[i] = 0
    for i in flow:
        for j in flow[i]:
            netflow[i] -= flow[i][j]
            netflow[j] += flow[i][j]
    return netflow
#print divergence(f).items()


The following code performs a sanity check on your function (but does not completely confirm correctness).

In [8]:
flow = flow_with_demands(G)
div = divergence(flow)
for i in G.nodes():
    print len (G.neighbors(i))
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

--------
2
3
2
4
5
2
6
3
3
4
4
3
4
3
6
8
5
6
2
5
6
6
3
4
6
3
5
5
4
6
5
4
6
3
1
5
6
5
5
8
4
3
4
4
4
2
7
4
6
Flow satisfies all demands: True
